In [1]:
import keras
import numpy as np
import tensorflow as tf
from keras import layers
from sklearn.metrics import classification_report
from keras import layers, models, regularizers
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau
import keras_tuner as kt
from tensorflow.keras.metrics import Precision, Recall
import tensorflow.keras.backend as K
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import torch
import torchaudio
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV, train_test_split

2024-07-04 18:36:09.772422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 18:36:09.877571: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 18:36:09.877817: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-04 18:36:09.997667: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import numpy as np
physical_devices = tf.config.list_physical_devices('GPU')


class AudioProcessor:
    def __init__(self, path_df, path_audio):
        self.path_df = "df_filter.csv"
        self.path_audio = path_audio
        self.load_data()

    def load_data(self):
        df = pd.read_csv(self.path_df)
        df["file_cut_path"] = self.path_audio + df["file_name"]
        df = df.reset_index(drop=True)
        self.dataset = df["file_cut_path"]

        self.log = []
        for i in range(len(df)):
            self.log.append(torch.load("torch_files14/" + f"torch_files14file{i}.pt"))

        self.y = df["label"]
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.log, self.y, random_state=42, stratify=self.y
        )

        self.process_tensors()

    def process_tensors(self):
        for i in range(len(self.X_train)):
            self.X_train[i] = self.X_train[i][0].detach().numpy()
            self.X_train[i] = np.concatenate(self.X_train[i])

        for i in range(len(self.X_test)):
            self.X_test[i] = self.X_test[i][0].detach().numpy()
            self.X_test[i] = np.concatenate(self.X_test[i])

2024-07-04 18:36:16.498189: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-04 18:36:16.503983: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-04 18:36:16.504030: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
processor = AudioProcessor(path_df="./df_filter.csv", path_audio="../audios/")
X_train   = processor.X_train
y_train  = processor.y_train
X_test = processor.X_test
y_test = processor.y_test

X_train_formated = np.array(X_train)
y_train_formated = np.array(y_train)
X_test_formated = np.array(X_test)
y_test_formated = np.array(y_test)

print("X_train_formated")
X_train_formated.shape
print("y_train_formated")
y_train_formated.shape
print("X_test_formated")
X_test_formated.shape
print("y_test_formated")
y_test_formated.shape


print("Datos cargados")

X_train_formated
y_train_formated
X_test_formated
y_test_formated
Datos cargados


In [ ]:
from imblearn.over_sampling import SMOTE

# Aplicar SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_formated, y_train_formated)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Calcular los pesos de las clases
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_resampled), y=y_train_resampled)
class_weights_dict = {0: class_weights[0], 1: class_weights[1]}

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Definir el modelo
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_resampled.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train_resampled, y_train_resampled,
                    validation_split=0.2,
                    epochs=100,
                    batch_size=32,
                    class_weight=class_weights_dict,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-07-03 20:44:01.464477: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-03 20:44:01.471708: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-03 20:44:01.471752: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have bee

Epoch 1/100


I0000 00:00:1720039451.614453   13845 service.cc:145] XLA service 0x7fe36c0093f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720039451.614531   13845 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1070, Compute Capability 6.1
2024-07-03 20:44:11.674069: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-03 20:44:11.866322: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906


16/53 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5036 - loss: 10.2967

I0000 00:00:1720039454.459146   13845 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


53/53 ━━━━━━━━━━━━━━━━━━━━ 8s 77ms/step - accuracy: 0.5248 - loss: 7.5127 - val_accuracy: 0.1520 - val_loss: 0.6993
Epoch 2/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5979 - loss: 0.7468 - val_accuracy: 0.1425 - val_loss: 0.7266
Epoch 3/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.6236 - loss: 0.7202 - val_accuracy: 0.1449 - val_loss: 0.7508
Epoch 4/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.5910 - loss: 0.7565 - val_accuracy: 0.1520 - val_loss: 0.7629
Epoch 5/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6115 - loss: 0.7009 - val_accuracy: 0.2090 - val_loss: 0.7524
Epoch 6/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6051 - loss: 0.7004 - val_accuracy: 0.1544 - val_loss: 0.7971
Epoch 7/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5916 - loss: 0.6787 - val_accuracy: 0.1710 - val_loss: 0.7987
Epoch 8/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5855 - loss: 0.7177 - val_accuracy: 0.1496 - val_l

In [8]:
from sklearn.metrics import precision_recall_curve

# Obtener probabilidades predichas
y_pred_proba = model.predict(X_test_formated).ravel()

# Calcular precisión y recall para varios umbrales
precision, recall, thresholds = precision_recall_curve(y_test_formated, y_pred_proba)

# Encontrar el umbral óptimo
optimal_threshold = thresholds[np.argmax(precision + recall)]

# Predecir con el umbral óptimo
y_pred = (y_pred_proba >= optimal_threshold).astype(int)

In [2]:
model.summary()

NameError: name 'model' is not defined

In [6]:
from sklearn import svm
model = svm.SVC(verbose=True,C= 1, gamma= 'scale', kernel= 'rbf',decision_function_shape='ovr')
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
score = accuracy_score(y_test, y_pred)
print(score)

[LibSVM].*
optimization finished, #iter = 1626
obj = -1130.185817, rho = 0.094821
nSV = 1643, nBSV = 1142
Total nSV = 1643
0.677257525083612


In [10]:
from sklearn.model_selection import cross_val_score

model = svm.SVC(C= 1, gamma= 'scale', kernel= 'rbf')
scores = cross_val_score(model, X_train, y_train, cv=5)
print("Puntajes de validación cruzada: ", scores)
print("Media de validación cruzada: ", scores.mean())

Puntajes de validación cruzada:  [0.66016713 0.65181058 0.62952646 0.63231198 0.61731844]
Media de validación cruzada:  0.6382269183486097
